## 2 Methods (as in point 2)

1. Use model trained on
    - current 2001, prediction_2003-2009, FEMP_2001, LIT_2001 to give output on 2011

2. Give it input as
    - current_2011, prediction_2011-2017 (obtained using model trained on 2011), FEMP_2011, LIT_2011 to give output on 2019
    - current_2011, prediction_2013-2019 (obtained using model trained on 2011), FEMP_2011, LIT_2011 to give output on 2019

In [2]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, accuracy_score

In [3]:
tss = pickle.load( open("/Users/arpitjain/Downloads/SatPRo/mlpros/classifier_cv/Temporal_Transferability/MSW_2011_TT_FEMP&LIT_base_XGB.pkl", 'rb'))
print('val_f1_score ',tss['xgBoost']['val_scores'])
print('train_f1_score ',tss['xgBoost']['train_scores'])

val_f1_score  [0.8270129911945027, 0.8159965598837557, 0.8113614817624573, 0.802237520193312, 0.8005558525591013]
train_f1_score  [0.9323896729776345, 0.9321812129150029, 0.9317655562340313, 0.9327571316386016, 0.9326533218886478]


In [4]:
n_estimators = tss['xgBoost']['specs'][0]['n_estimators']
max_depth = tss['xgBoost']['specs'][0]['max_depth']
learning_rate = tss['xgBoost']['specs'][0]['learning_rate']
objective = tss['xgBoost']['specs'][0]['objective']
booster = tss['xgBoost']['specs'][0]['booster']
gamma = tss['xgBoost']['specs'][0]['gamma']
min_child_weight = tss['xgBoost']['specs'][0]['min_child_weight']
max_delta_step = tss['xgBoost']['specs'][0]['max_delta_step']
subsample = tss['xgBoost']['specs'][0]['subsample']
colsample_bytree = tss['xgBoost']['specs'][0]['colsample_bytree']
colsample_bylevel = tss['xgBoost']['specs'][0]['colsample_bylevel']
colsample_bynode = tss['xgBoost']['specs'][0]['colsample_bynode']
reg_alpha = tss['xgBoost']['specs'][0]['reg_alpha']
reg_lambda = tss['xgBoost']['specs'][0]['reg_lambda']
scale_pos_weight = tss['xgBoost']['specs'][0]['scale_pos_weight']
base_score = tss['xgBoost']['specs'][0]['base_score']

n_splits = tss['xgBoost']['specs'][0]['kFold_splits']

print('                 ')
print('Re-Training again for calculating f1 and train scores')

xgbc = XGBClassifier(n_estimators=n_estimators, 
         max_depth=max_depth, learning_rate=learning_rate, 
         objective=objective, booster=booster,n_jobs=-1, 
         gamma=gamma, min_child_weight=min_child_weight, 
         max_delta_step=max_delta_step, subsample=subsample, 
         colsample_bytree=colsample_bytree, 
         colsample_bylevel=colsample_bylevel, colsample_bynode=colsample_bynode, 
         reg_alpha=reg_alpha, reg_lambda=reg_lambda, 
         scale_pos_weight=scale_pos_weight, base_score=base_score, random_state=0)

cv = KFold(n_splits=n_splits, shuffle=True)

                 
Re-Training again for calculating f1 and train scores


In [5]:
ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/District - Ground Truth - 2011&2001.csv")
femp_lit = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/FEMP&LIT.csv")
cols_2 = ['census_code', 'LIT_2001', 'FEMP_2001']
femp_lit = femp_lit[cols_2]

indicator = 'MSW'

cols = ['census_code', indicator+'_2001', indicator+'_2011']
gt = ground_truth[cols]

df = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data/MSW_CC.csv")

data = gt.merge(df,on='census_code', how='left')
data = femp_lit.merge(data, on='census_code', how='left')
data.dropna(inplace=True)

In [6]:
data.head(5)

,census_code,LIT_2001,FEMP_2001,MSW_2001,MSW_2011,predictions_2001,predictions_2003,predictions_2005,predictions_2007,predictions_2009,predictions_2011
0,1,1,2,1,1,1.0,1.0,3.0,3.0,3.0,3.0
1,2,1,2,3,3,3.0,3.0,3.0,3.0,3.0,3.0
3,4,1,3,1,3,1.0,1.0,1.0,1.0,1.0,1.0
4,5,1,2,1,1,1.0,3.0,3.0,3.0,3.0,3.0
5,6,1,2,1,1,1.0,3.0,3.0,3.0,3.0,3.0


In [7]:
feature_cols = [indicator+'_2001','LIT_2001','FEMP_2001','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009']
target = indicator+'_2011'

In [8]:
X = data[feature_cols].values
y = data[target].values

In [9]:
val_f1score = []
val_accscore = []
train_f1score = []
train_accscore = []
# f1score = []
# accscore = []
counter=0
for train_index, test_index in cv.split(X):
    
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    
    best_xgb = xgbc.fit(X_train, y_train)
    
#     prediction_2011 = best_xgb.predict(X_2011)
#     f1_weight = f1_score(y_2011, prediction_2011, average='weighted')
#     acc = accuracy_score(y_2011, prediction_2011)

#     f1score.append(f1_weight)
#     accscore.append(acc)

    predictions = best_xgb.predict(X_test)
    predictions_train = best_xgb.predict(X_train)

    f1_weight = f1_score(y_test, predictions, average='weighted')
    f1_weight_train = f1_score(y_train, predictions_train, average='weighted')
    acc = accuracy_score(y_test, predictions)
    acc_train = accuracy_score(y_train, predictions_train)
    print(counter, f1_weight, f1_weight_train, acc, acc_train)
    counter=counter+1

    val_f1score.append(f1_weight)
    train_f1score.append(f1_weight_train)
    val_accscore.append(acc)
    train_accscore.append(acc_train)

val_f1score = np.array(val_f1score).mean()
train_f1score = np.array(train_f1score).mean()
val_accscore = np.array(val_accscore).mean()
train_accscore = np.array(train_accscore).mean()
print('------')
print(val_f1score, train_f1score, val_accscore, train_accscore)

0 0.8363721664214275 0.9327865673880659 0.8275862068965517 0.9330218068535826
1 0.8191615363914536 0.9326850460935188 0.8189655172413793 0.9327073552425665
2 0.7969480661454986 0.9364927805058052 0.8017241379310345 0.9365853658536586
3 0.8468013730691214 0.9257470558035039 0.8448275862068966 0.9261006289308176
4 0.8023506724095454 0.9361304871555528 0.8017241379310345 0.9363636363636364
------
0.8203267628874092 0.9327683873892892 0.8189655172413793 0.9329557586488523


In [9]:
# Model has been trained now

In [10]:
file = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data_2011-19/MSW_CC.csv")

In [11]:
file.columns

Index(['census_code', 'predictions_2011', 'predictions_2013',
       'predictions_2015', 'predictions_2017', 'predictions_2019'],
      dtype='object')

In [12]:
ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/District - Ground Truth - 2011&2001.csv")
gt_cols = ['census_code','MSW_2011']

ground_truth = ground_truth[gt_cols]
femp_lit = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/FEMP&LIT.csv")
cols_2 = ['census_code', 'LIT_2011', 'FEMP_2011']
femp_lit = femp_lit[cols_2]

In [13]:
print(ground_truth.shape)
print(femp_lit.shape)
gt_femlit = ground_truth.merge(femp_lit, on='census_code', how='left')
gt_femlit.dropna(inplace=True)
print(gt_femlit.shape)

(593, 2)
(593, 3)
(593, 4)


In [14]:
X_test_fut = gt_femlit.merge(file, on='census_code', how='left')
X_test_fut.dropna(inplace=True)
X_test_fut.shape

(589, 9)

In [15]:
X_test_fut.head()

,census_code,MSW_2011,LIT_2011,FEMP_2011,predictions_2011,predictions_2013,predictions_2015,predictions_2017,predictions_2019
0,1,1,1,2,1.0,3.0,3.0,3.0,3.0
1,2,3,1,2,3.0,3.0,3.0,1.0,3.0
2,3,1,2,3,1.0,3.0,3.0,3.0,3.0
3,4,3,2,3,3.0,3.0,3.0,3.0,3.0
4,5,1,1,2,1.0,3.0,1.0,1.0,1.0


In [16]:
X_test_fut.columns

Index(['census_code', 'MSW_2011', 'LIT_2011', 'FEMP_2011', 'predictions_2011',
       'predictions_2013', 'predictions_2015', 'predictions_2017',
       'predictions_2019'],
      dtype='object')

In [17]:
census_codes = X_test_fut['census_code'].values

In [18]:
# NOTE that order should be same

cols_fut = ['MSW_2011', 'LIT_2011', 'FEMP_2011','predictions_2011', 'predictions_2013', 'predictions_2015','predictions_2017']
X_test_fut = X_test_fut[cols_fut]

In [19]:
X_test_fut = X_test_fut.values

In [20]:
X_test_fut.shape

(589, 7)

In [21]:
X_train.shape

(660, 7)

In [22]:
predictions_future = best_xgb.predict(X_test_fut)

In [23]:
predictions_future.shape

(589,)

In [24]:
census_codes.shape

(589,)

In [25]:
final_predictions = np.stack((census_codes, predictions_future))

In [26]:
final_predictions.shape

(2, 589)

In [27]:
final_predictions_df = pd.DataFrame(final_predictions)

In [28]:
final_predictions_df = final_predictions_df.transpose()

In [29]:
final_predictions_df.columns = ['census_code', 'prediction_2019']

In [30]:
final_predictions_df.head()

,census_code,prediction_2019
0,1,1
1,2,3
2,3,1
3,4,3
4,5,2


In [31]:
final_predictions_df['prediction_2019'].value_counts()

3    263
2    247
1     79
Name: prediction_2019, dtype: int64

In [32]:
final_predictions_df.to_csv('Future_MSW_2019_M1.csv', index=False)